Load Dataset

In [1]:
import timeit
start_time = timeit.default_timer()

def print_elapsed_time():
    start_time = timeit.default_timer()
    
def start_timer():
    elapsed = (timeit.default_timer() - start_time)/60
    print(elapsed)

In [2]:
from sklearn.datasets import load_files   
from keras.utils import np_utils
import numpy as np
from glob import glob




def load_dataset(data_path, shuffle=None):
    kwargs = {}
    if shuffle != None:
        kwargs['shuffle'] = shuffle
    data = load_files(data_path, **kwargs)
    img_files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return img_files, targets



start_timer()
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test', shuffle=False)

# load lables
label_name = [item[11:-1] for item in sorted(glob("data/train/*/"))]

print_elapsed_time()

C:\Users\vinayrraj\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0.3811088810837933


In [3]:
start_timer()

print('train_files size: {}'.format(len(train_files)))
print('train_files shape: {}'.format(train_files.shape))
print('target shape: {}'.format(train_targets.shape))
print(label_name)

start_time = timeit.default_timer()
elapsed = (timeit.default_timer() - start_time)/60
print(elapsed)


print_elapsed_time()


5.190188388953173
train_files size: 2000
train_files shape: (2000,)
target shape: (2000, 3)
['melanoma', 'nevus', 'seborrheic_keratosis']
7.765273693394192e-07


In [4]:
start_timer()
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from tqdm import tqdm


def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(384, 256))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(image_paths):
    return np.vstack([path_to_tensor(path) for path in image_paths])

print_elapsed_time()


0.0003094760232234724


Transfer Learning and putting images into tensor.

In [5]:
start_timer()

train_tensors = paths_to_tensor(tqdm(train_files))
valid_tensors = paths_to_tensor(tqdm(valid_files))
test_tensors = paths_to_tensor(tqdm(test_files))

print(train_tensors.shape)

print_elapsed_time()


0.0024611650982516646


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:25<00:00,  1.84it/s]


(2000, 384, 256, 3)


Flip Images

In [6]:
start_timer()


import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator


apply_train_image_transform = False

if apply_train_image_transform:
    # Caution: Doesn't guarantee prevention of duplication.
    datagen_train = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True)
    
    datagen_train.fit(train_tensors)
    shape = (train_tensors.shape[0] * 2,) + train_tensors.shape[1:]
    generated = np.ndarray(shape=shape)
    for i, image in tqdm(enumerate(train_tensors)):
        generated[i] = datagen_train.random_transform(image)
    
    train_tensors = np.concatenate((train_tensors, generated))
    train_targets = train_targets.repeat(2, axis=0)
    
print_elapsed_time()


12.656683204139657


Transfer learning using Inception Resnet V2¶

In [7]:
start_timer()

train_imgs_preprocess = preprocess_input(train_tensors)
valid_imgs_preprocess = preprocess_input(valid_tensors)
test_imgs_preprocess = preprocess_input(test_tensors)
del train_tensors, valid_tensors, test_tensors

print_elapsed_time()


12.681337163063922


In [8]:
start_timer()

from keras.applications.inception_resnet_v2 import InceptionResNetV2
transfer_model = InceptionResNetV2(include_top=False)

train_data = transfer_model.predict(train_imgs_preprocess)
valid_data = transfer_model.predict(valid_imgs_preprocess)
test_data = transfer_model.predict(test_imgs_preprocess)

del train_imgs_preprocess, valid_imgs_preprocess, test_imgs_preprocess
print(train_data.shape)

print_elapsed_time()


12.714697982052833
(2000, 10, 6, 1536)


In [9]:
start_timer()

from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
my_model.add(Dropout(0.2))
my_model.add(Dense(1024, activation='relu'))
my_model.add(Dropout(0.2))
my_model.add(Dense(3, activation='softmax'))
my_model.summary()

print_elapsed_time()


67.14202191513837
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1573888   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 1,576,963
Trainable params: 1,576,963
Non-trainable params: 0
_________________________________________________________________


In [10]:
start_timer()

my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print_elapsed_time()


67.14523349580675


In [11]:
start_timer()

from keras.callbacks import ModelCheckpoint


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_data, train_targets, 
          validation_data=(valid_data, valid_targets),
          epochs=70, batch_size=200, callbacks=[my_checkpointer], verbose=1)

print_elapsed_time()


67.14736855354285
Train on 2000 samples, validate on 150 samples
Epoch 1/70
2000/2000 [==============================] - ETA: 29s - loss: 1.0736 - acc: 0.48 - ETA: 13s - loss: 2.4214 - acc: 0.58 - ETA: 8s - loss: 2.2929 - acc: 0.6283 - ETA: 5s - loss: 2.9955 - acc: 0.530 - ETA: 3s - loss: 2.9371 - acc: 0.517 - ETA: 2s - loss: 2.9326 - acc: 0.549 - ETA: 1s - loss: 3.0722 - acc: 0.565 - ETA: 1s - loss: 3.1481 - acc: 0.583 - ETA: 0s - loss: 3.2026 - acc: 0.595 - 5s 3ms/step - loss: 3.2636 - acc: 0.6035 - val_loss: 5.6274 - val_acc: 0.5200

Epoch 00001: val_loss improved from inf to 5.62745, saving model to saved_models/weights.best.my.hdf5
Epoch 2/70
2000/2000 [==============================] - ETA: 1s - loss: 3.3109 - acc: 0.685 - ETA: 1s - loss: 2.9709 - acc: 0.697 - ETA: 0s - loss: 2.7767 - acc: 0.681 - ETA: 0s - loss: 2.7757 - acc: 0.605 - ETA: 0s - loss: 2.7267 - acc: 0.558 - ETA: 0s - loss: 2.6732 - acc: 0.570 - ETA: 0s - loss: 2.6565 - acc: 0.578 - ETA: 0s - loss: 2.6858 - acc: 0.5

2000/2000 [==============================] - ETA: 1s - loss: 0.5771 - acc: 0.750 - ETA: 1s - loss: 0.5981 - acc: 0.750 - ETA: 0s - loss: 0.6088 - acc: 0.740 - ETA: 0s - loss: 0.5935 - acc: 0.742 - ETA: 0s - loss: 0.5919 - acc: 0.745 - ETA: 0s - loss: 0.5796 - acc: 0.751 - ETA: 0s - loss: 0.5898 - acc: 0.747 - ETA: 0s - loss: 0.5876 - acc: 0.749 - ETA: 0s - loss: 0.5879 - acc: 0.748 - 1s 716us/step - loss: 0.5845 - acc: 0.7485 - val_loss: 0.7530 - val_acc: 0.6667

Epoch 00015: val_loss improved from 0.78672 to 0.75301, saving model to saved_models/weights.best.my.hdf5
Epoch 16/70
2000/2000 [==============================] - ETA: 1s - loss: 0.5846 - acc: 0.755 - ETA: 1s - loss: 0.5938 - acc: 0.740 - ETA: 0s - loss: 0.5792 - acc: 0.760 - ETA: 0s - loss: 0.5782 - acc: 0.757 - ETA: 0s - loss: 0.5670 - acc: 0.766 - ETA: 0s - loss: 0.5708 - acc: 0.765 - ETA: 0s - loss: 0.5746 - acc: 0.763 - ETA: 0s - loss: 0.5796 - acc: 0.765 - ETA: 0s - loss: 0.5797 - acc: 0.767 - 1s 726us/step - loss: 0.581


Epoch 00029: val_loss did not improve from 0.68532
Epoch 30/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4574 - acc: 0.820 - ETA: 1s - loss: 0.4677 - acc: 0.825 - ETA: 0s - loss: 0.5081 - acc: 0.803 - ETA: 0s - loss: 0.4987 - acc: 0.800 - ETA: 0s - loss: 0.5130 - acc: 0.791 - ETA: 0s - loss: 0.5048 - acc: 0.793 - ETA: 0s - loss: 0.5169 - acc: 0.788 - ETA: 0s - loss: 0.5117 - acc: 0.789 - ETA: 0s - loss: 0.5071 - acc: 0.792 - 1s 725us/step - loss: 0.5060 - acc: 0.7905 - val_loss: 0.7048 - val_acc: 0.7000

Epoch 00030: val_loss did not improve from 0.68532
Epoch 31/70
2000/2000 [==============================] - ETA: 1s - loss: 0.5407 - acc: 0.800 - ETA: 1s - loss: 0.5220 - acc: 0.787 - ETA: 0s - loss: 0.5078 - acc: 0.783 - ETA: 0s - loss: 0.5041 - acc: 0.785 - ETA: 0s - loss: 0.4959 - acc: 0.793 - ETA: 0s - loss: 0.5086 - acc: 0.785 - ETA: 0s - loss: 0.4915 - acc: 0.796 - ETA: 0s - loss: 0.4920 - acc: 0.792 - ETA: 0s - loss: 0.4904 - acc: 0.795 - 1s 724us/step - lo

2000/2000 [==============================] - ETA: 1s - loss: 0.3691 - acc: 0.845 - ETA: 1s - loss: 0.4107 - acc: 0.827 - ETA: 0s - loss: 0.4171 - acc: 0.830 - ETA: 0s - loss: 0.4253 - acc: 0.821 - ETA: 0s - loss: 0.4225 - acc: 0.825 - ETA: 0s - loss: 0.4181 - acc: 0.830 - ETA: 0s - loss: 0.4194 - acc: 0.829 - ETA: 0s - loss: 0.4217 - acc: 0.830 - ETA: 0s - loss: 0.4276 - acc: 0.826 - 1s 724us/step - loss: 0.4277 - acc: 0.8280 - val_loss: 0.7046 - val_acc: 0.7000

Epoch 00045: val_loss did not improve from 0.67479
Epoch 46/70
2000/2000 [==============================] - ETA: 1s - loss: 0.4223 - acc: 0.840 - ETA: 1s - loss: 0.4613 - acc: 0.810 - ETA: 0s - loss: 0.4563 - acc: 0.815 - ETA: 0s - loss: 0.4437 - acc: 0.818 - ETA: 0s - loss: 0.4520 - acc: 0.814 - ETA: 0s - loss: 0.4521 - acc: 0.815 - ETA: 0s - loss: 0.4480 - acc: 0.820 - ETA: 0s - loss: 0.4411 - acc: 0.825 - ETA: 0s - loss: 0.4386 - acc: 0.828 - 1s 728us/step - loss: 0.4354 - acc: 0.8285 - val_loss: 0.7702 - val_acc: 0.7067

E

2000/2000 [==============================] - ETA: 1s - loss: 0.3538 - acc: 0.855 - ETA: 1s - loss: 0.3658 - acc: 0.852 - ETA: 0s - loss: 0.3665 - acc: 0.858 - ETA: 0s - loss: 0.3756 - acc: 0.851 - ETA: 0s - loss: 0.3807 - acc: 0.843 - ETA: 0s - loss: 0.3794 - acc: 0.840 - ETA: 0s - loss: 0.3763 - acc: 0.843 - ETA: 0s - loss: 0.3728 - acc: 0.846 - ETA: 0s - loss: 0.3662 - acc: 0.850 - 1s 719us/step - loss: 0.3701 - acc: 0.8480 - val_loss: 0.7948 - val_acc: 0.6933

Epoch 00060: val_loss did not improve from 0.67479
Epoch 61/70
2000/2000 [==============================] - ETA: 1s - loss: 0.3597 - acc: 0.850 - ETA: 1s - loss: 0.3443 - acc: 0.875 - ETA: 0s - loss: 0.3580 - acc: 0.868 - ETA: 0s - loss: 0.3734 - acc: 0.861 - ETA: 0s - loss: 0.3763 - acc: 0.859 - ETA: 0s - loss: 0.3622 - acc: 0.861 - ETA: 0s - loss: 0.3598 - acc: 0.857 - ETA: 0s - loss: 0.3501 - acc: 0.858 - ETA: 0s - loss: 0.3548 - acc: 0.855 - 1s 715us/step - loss: 0.3547 - acc: 0.8555 - val_loss: 0.7081 - val_acc: 0.7000

E

In [12]:
start_timer()

my_model.load_weights(checkpoint_filepath)

print_elapsed_time()


68.97895079205792


Evaluavate 

In [13]:
start_timer()

import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_data]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_transfer.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])
        
print_elapsed_time()


68.99812342347877


Category 1 Score: 0.526
Category 2 Score: 0.606
Category 3 Score: 0.566

![ROC Curves](images/figure_1.png)

Without transfer learning, loading images into tensor

In [14]:
start_timer()
train_tensors = paths_to_tensor(tqdm(train_files))
train_tensors = train_tensors.astype('float32') / 255

valid_tensors = paths_to_tensor(tqdm(valid_files))
valid_tensors = valid_tensors.astype('float32') / 255

test_tensors = paths_to_tensor(tqdm(test_files))
test_tensors = test_tensors.astype('float32') / 255

print(train_tensors.shape)

print_elapsed_time()

69.04965888261978


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:33<00:00,  1.80it/s]


(2000, 384, 256, 3)


In [15]:
start_timer()
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', 
                        input_shape=train_tensors.shape[1:]))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))

my_model.add(Conv2D(filters=2048, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))


my_model.add(GlobalAveragePooling2D())

my_model.add(Dense(3, activation='softmax'))

my_model.summary()

print_elapsed_time()

82.24814145427362
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_204 (Conv2D)          (None, 384, 256, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 192, 128, 16)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 192, 128, 16)      0         
_________________________________________________________________
conv2d_205 (Conv2D)          (None, 192, 128, 64)      9280      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 96, 64, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 96, 64, 64)        0         
_________________________________________________________________
conv2d_206 (Conv2D)          (None, 96, 64, 256)       147

In [16]:
start_timer()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print_elapsed_time()

82.25484451532746


In [17]:
start_timer()


from keras.callbacks import ModelCheckpoint
import os


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=40, batch_size=100, callbacks=[my_checkpointer], verbose=1)


print_elapsed_time()

82.25640431134822
Train on 2000 samples, validate on 150 samples
Epoch 1/40
2000/2000 [==============================] - ETA: 27:31 - loss: 1.0868 - acc: 0.36 - ETA: 25:09 - loss: 3.0417 - acc: 0.52 - ETA: 23:32 - loss: 3.4784 - acc: 0.59 - ETA: 21:53 - loss: 4.0997 - acc: 0.60 - ETA: 20:27 - loss: 4.0212 - acc: 0.63 - ETA: 18:59 - loss: 3.9957 - acc: 0.65 - ETA: 17:35 - loss: 4.0236 - acc: 0.66 - ETA: 16:15 - loss: 4.1855 - acc: 0.66 - ETA: 14:51 - loss: 4.3473 - acc: 0.66 - ETA: 13:31 - loss: 4.4928 - acc: 0.66 - ETA: 12:10 - loss: 4.6119 - acc: 0.66 - ETA: 10:50 - loss: 4.6439 - acc: 0.66 - ETA: 9:28 - loss: 4.7083 - acc: 0.6638 - ETA: 8:06 - loss: 4.7749 - acc: 0.662 - ETA: 6:45 - loss: 4.7574 - acc: 0.666 - ETA: 5:24 - loss: 4.7522 - acc: 0.669 - ETA: 4:02 - loss: 4.7951 - acc: 0.668 - ETA: 2:41 - loss: 4.7525 - acc: 0.673 - ETA: 1:20 - loss: 4.7569 - acc: 0.674 - 1650s 825ms/step - loss: 4.8253 - acc: 0.6720 - val_loss: 7.7367 - val_acc: 0.5200

Epoch 00001: val_loss improved fro

2000/2000 [==============================] - ETA: 25:18 - loss: 4.9966 - acc: 0.69 - ETA: 24:36 - loss: 5.5607 - acc: 0.65 - ETA: 23:05 - loss: 4.9966 - acc: 0.69 - ETA: 21:41 - loss: 5.0369 - acc: 0.68 - ETA: 20:23 - loss: 5.0933 - acc: 0.68 - ETA: 19:00 - loss: 4.9966 - acc: 0.69 - ETA: 17:38 - loss: 4.9506 - acc: 0.69 - ETA: 16:15 - loss: 5.0168 - acc: 0.68 - ETA: 14:53 - loss: 5.0324 - acc: 0.68 - ETA: 13:32 - loss: 5.0450 - acc: 0.68 - ETA: 12:12 - loss: 4.9966 - acc: 0.69 - ETA: 10:49 - loss: 4.9295 - acc: 0.69 - ETA: 9:27 - loss: 4.9098 - acc: 0.6954 - ETA: 8:06 - loss: 4.9390 - acc: 0.693 - ETA: 6:45 - loss: 4.9429 - acc: 0.693 - ETA: 5:23 - loss: 4.9865 - acc: 0.690 - ETA: 4:02 - loss: 4.9682 - acc: 0.691 - ETA: 2:41 - loss: 4.9787 - acc: 0.691 - ETA: 1:21 - loss: 5.0645 - acc: 0.685 - 1660s 830ms/step - loss: 5.0611 - acc: 0.6860 - val_loss: 7.7367 - val_acc: 0.5200

Epoch 00009: val_loss did not improve from 7.73669
Epoch 10/40
2000/2000 [==============================] - ET

2000/2000 [==============================] - ETA: 26:12 - loss: 4.9966 - acc: 0.69 - ETA: 24:38 - loss: 4.9966 - acc: 0.69 - ETA: 23:14 - loss: 4.6742 - acc: 0.71 - ETA: 21:49 - loss: 4.9160 - acc: 0.69 - ETA: 20:22 - loss: 5.2545 - acc: 0.67 - ETA: 19:01 - loss: 5.2652 - acc: 0.67 - ETA: 17:41 - loss: 5.0196 - acc: 0.68 - ETA: 16:17 - loss: 5.0571 - acc: 0.68 - ETA: 14:56 - loss: 5.1041 - acc: 0.68 - ETA: 13:33 - loss: 5.1900 - acc: 0.67 - ETA: 12:13 - loss: 5.1871 - acc: 0.67 - ETA: 10:50 - loss: 5.1309 - acc: 0.68 - ETA: 9:29 - loss: 5.1454 - acc: 0.6808 - ETA: 8:08 - loss: 5.1348 - acc: 0.681 - ETA: 6:47 - loss: 5.1793 - acc: 0.678 - ETA: 5:26 - loss: 5.1477 - acc: 0.680 - ETA: 4:04 - loss: 5.1483 - acc: 0.680 - ETA: 2:43 - loss: 5.0593 - acc: 0.686 - ETA: 1:21 - loss: 5.0560 - acc: 0.686 - 1669s 835ms/step - loss: 5.0611 - acc: 0.6860 - val_loss: 7.7367 - val_acc: 0.5200

Epoch 00017: val_loss did not improve from 7.73669
Epoch 18/40
2000/2000 [==============================] - ET

2000/2000 [==============================] - ETA: 25:43 - loss: 5.4802 - acc: 0.66 - ETA: 24:05 - loss: 5.2384 - acc: 0.67 - ETA: 22:54 - loss: 5.3727 - acc: 0.66 - ETA: 21:31 - loss: 5.5204 - acc: 0.65 - ETA: 20:22 - loss: 5.3512 - acc: 0.66 - ETA: 18:56 - loss: 5.1578 - acc: 0.68 - ETA: 17:35 - loss: 5.1348 - acc: 0.68 - ETA: 16:17 - loss: 5.0973 - acc: 0.68 - ETA: 14:54 - loss: 5.0324 - acc: 0.68 - ETA: 13:32 - loss: 4.9805 - acc: 0.69 - ETA: 12:10 - loss: 5.0845 - acc: 0.68 - ETA: 10:50 - loss: 5.1175 - acc: 0.68 - ETA: 9:28 - loss: 5.1330 - acc: 0.6815 - ETA: 8:06 - loss: 5.0887 - acc: 0.684 - ETA: 6:46 - loss: 5.0826 - acc: 0.684 - ETA: 5:25 - loss: 5.0772 - acc: 0.685 - ETA: 4:04 - loss: 5.0819 - acc: 0.684 - ETA: 2:42 - loss: 5.0951 - acc: 0.683 - ETA: 1:21 - loss: 5.0814 - acc: 0.684 - 1662s 831ms/step - loss: 5.0611 - acc: 0.6860 - val_loss: 7.7367 - val_acc: 0.5200

Epoch 00025: val_loss did not improve from 7.73669
Epoch 26/40
2000/2000 [==============================] - ET

2000/2000 [==============================] - ETA: 25:29 - loss: 5.1578 - acc: 0.68 - ETA: 24:12 - loss: 5.0772 - acc: 0.68 - ETA: 22:56 - loss: 4.8892 - acc: 0.69 - ETA: 21:42 - loss: 4.7145 - acc: 0.70 - ETA: 20:17 - loss: 4.6098 - acc: 0.71 - ETA: 18:59 - loss: 4.4862 - acc: 0.72 - ETA: 17:40 - loss: 4.4210 - acc: 0.72 - ETA: 16:14 - loss: 4.5534 - acc: 0.71 - ETA: 14:51 - loss: 4.6742 - acc: 0.71 - ETA: 13:28 - loss: 4.7548 - acc: 0.70 - ETA: 12:08 - loss: 4.7768 - acc: 0.70 - ETA: 10:46 - loss: 4.7951 - acc: 0.70 - ETA: 9:25 - loss: 4.8478 - acc: 0.6992 - ETA: 8:03 - loss: 4.9506 - acc: 0.692 - ETA: 6:43 - loss: 5.0074 - acc: 0.689 - ETA: 5:23 - loss: 5.0369 - acc: 0.687 - ETA: 4:02 - loss: 4.9776 - acc: 0.691 - ETA: 2:41 - loss: 5.0056 - acc: 0.689 - ETA: 1:20 - loss: 5.0645 - acc: 0.685 - 1647s 824ms/step - loss: 5.0611 - acc: 0.6860 - val_loss: 7.7367 - val_acc: 0.5200

Epoch 00033: val_loss did not improve from 7.73669
Epoch 34/40
2000/2000 [==============================] - ET

In [18]:
start_timer()

my_model.load_weights(checkpoint_filepath)

print_elapsed_time()

1185.5332182366174


In [19]:
start_timer()


import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_tensors]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_cnn.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])
        

print_elapsed_time()

1185.5665660679726
